In [19]:
import json
import os
import cv2
import pandas as pd
import numpy as np
from skimage.feature import hog
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import joblib
from imutils.object_detection import non_max_suppression
from sklearn.metrics import accuracy_score


In [ ]:
def load_coco_annotations(json_path, img_dir):
    with open(json_path, "r") as f:
        coco_data = json.load(f)
    
    images = {img["id"]: img["file_name"] for img in coco_data["images"]}
    annotations = coco_data["annotations"] 
    
    data = []
    for ann in annotations:
        image_id = ann["image_id"]
        image_path = os.path.join(img_dir, images[image_id])
        category_id = ann["category_id"]  # Giả sử đây là nhãn cho người
        bbox = ann["bbox"]  # [x, y, width, height]

        # Đọc ảnh và cắt vùng bounding box của người
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue  # Bỏ qua ảnh lỗi

        x, y, w, h = map(int, bbox)
        cropped_img = img[y:y+h, x:x+w]
        cropped_img = cv2.resize(cropped_img, (64, 128))

        # Trích xuất đặc trưng HOG cho vùng người
        features = hog(cropped_img, pixels_per_cell=(4, 4), cells_per_block=(2, 2))
        
        data.append({
            "image_path": image_path,
            "features": features,
            "label": category_id  # Nhãn người (ví dụ: 1)
        })
    
    return pd.DataFrame(data)

def extract_negative_samples(json_path, img_dir, num_samples=10):
    """
    Trích xuất các mẫu âm từ các vùng không chứa bounding box của người.
    num_samples: số lượng mẫu âm cần lấy cho mỗi ảnh.
    """
    with open(json_path, "r") as f:
        coco_data = json.load(f)
    
    images = {img["id"]: img["file_name"] for img in coco_data["images"]}
    # Tập hợp tất cả các bounding box của người cho mỗi ảnh
    bbox_dict = {}
    for ann in coco_data["annotations"]:
        image_id = ann["image_id"]
        bbox = ann["bbox"]  # [x, y, width, height]
        if image_id not in bbox_dict:
            bbox_dict[image_id] = []
        bbox_dict[image_id].append(list(map(int, bbox)))
    
    negative_data = []
    for image_id, file_name in images.items():
        image_path = os.path.join(img_dir, file_name)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        img_h, img_w = img.shape
        
        # Lấy các bounding box của người (nếu có)
        person_bboxes = bbox_dict.get(image_id, [])
        
        # Sinh ngẫu nhiên các vùng mẫu âm
        samples = 0
        attempts = 0
        while samples < num_samples and attempts < num_samples * 10:
            # Sinh tọa độ ngẫu nhiên
            w, h = 64, 128
            x = np.random.randint(0, img_w - w)
            y = np.random.randint(0, img_h - h)
            candidate_box = [x, y, x+w, y+h]  # [x1, y1, x2, y2]
            
            # Kiểm tra giao nhau với các bounding box của người
            overlap = False
            for bbox in person_bboxes:
                # bbox gốc ở dạng [x, y, width, height] chuyển thành [x1, y1, x2, y2]
                pb = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
                # Tính giao nhau
                ix1 = max(candidate_box[0], pb[0])
                iy1 = max(candidate_box[1], pb[1])
                ix2 = min(candidate_box[2], pb[2])
                iy2 = min(candidate_box[3], pb[3])
                iw = max(0, ix2 - ix1)
                ih = max(0, iy2 - iy1)
                if iw * ih > 0:  # Nếu có giao nhau
                    overlap = True
                    break
            if not overlap:
                cropped_img = img[y:y+h, x:x+w]
                # Nếu kích thước không đủ thì bỏ qua
                if cropped_img.shape[0] != h or cropped_img.shape[1] != w:
                    attempts += 1
                    continue
                cropped_img = cv2.resize(cropped_img, (64, 128))
                features = hog(cropped_img, pixels_per_cell=(4, 4), cells_per_block=(2, 2))
                negative_data.append({
                    "image_path": image_path,
                    "features": features,
                    "label": 0  # Nhãn cho non-person
                })
                samples += 1
            attempts += 1
    return pd.DataFrame(negative_data)

# Ví dụ: Load tập train & test cho mẫu người và mẫu non-person
train_person_df = load_coco_annotations("train/_annotations.coco.json", "train/")
train_negative_df = extract_negative_samples("train/_annotations.coco.json", "train/", num_samples=5)

test_person_df = load_coco_annotations("test/_annotations.coco.json", "test/")
test_negative_df = extract_negative_samples("test/_annotations.coco.json", "test/", num_samples=5)

# Kết hợp dữ liệu
train_df = pd.concat([train_person_df, train_negative_df]).reset_index(drop=True)
test_df = pd.concat([test_person_df, test_negative_df]).reset_index(drop=True)


In [21]:
# Chuẩn bị dữ liệu
X_train = np.array(train_df["features"].tolist(), dtype=np.float32)
y_train = train_df["label"].values
X_test = np.array(test_df["features"].tolist(), dtype=np.float32)
y_test = test_df["label"].values

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)  # Chuyển về float32
X_test_scaled = scaler.transform(X_test).astype(np.float32)  # Chuyển về float32

# Lưu scaler
joblib.dump(scaler, "scaler.pkl")

# Huấn luyện Random Forest
rf = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)
rf.fit(X_train_scaled, y_train)
y_pred = rf.predict(X_test_scaled)

# Tính Accuracy
accuracy = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f"🎯 Accuracy: {accuracy:.4f}")


🎯 Accuracy: 0.9877


In [22]:
print(y_train)

[1 1 1 ... 0 0 0]


In [23]:

print(accuracy)

0.9876543209876543


In [24]:
# Lưu mô hình
joblib.dump(rf, "random_forest_model.pkl")
print("✅ Mô hình Random Forest đã được huấn luyện!")

✅ Mô hình Random Forest đã được huấn luyện!


In [25]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(32907, 16740)
(3321, 16740)
(32907,)
(3321,)


In [26]:
# Hàm phát hiện người với Sliding Window & HOG
def detect_people(image_path, model, scaler):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Lỗi: Không thể đọc ảnh {image_path}")
        return []
    
    h, w = img.shape
    window_size = (64, 128)
    step_size = 16
    detections = []
    
    for y in range(0, h - window_size[1], step_size):
        for x in range(0, w - window_size[0], step_size):
            window = img[y:y+window_size[1], x:x+window_size[0]]
            features = hog(window, pixels_per_cell=(4, 4), cells_per_block=(2, 2))
            features_scaled = scaler.transform([features])
            pred = model.predict(features_scaled)
            if pred == 1:
                detections.append((x, y, x+window_size[0], y+window_size[1]))
    
    # Loại bỏ vùng trùng lặp bằng Non-Maximum Suppression (NMS)
    detections = np.array(detections)
    if len(detections) > 0:
        picks = non_max_suppression(detections, probs=None, overlapThresh=0.3)
        return picks.tolist()
    return []



In [27]:
print(test_df)

                                             image_path  \
0     test/11516_jpg.rf.7cf4e332eafdb31672c0025195b6...   
1     test/11516_jpg.rf.7cf4e332eafdb31672c0025195b6...   
2     test/12545_jpg.rf.69333f3b7d11b9cce33062b0cacf...   
3     test/6552_jpg.rf.ad79089b0fb56be23f73253fc5c0a...   
4     test/6552_jpg.rf.ad79089b0fb56be23f73253fc5c0a...   
...                                                 ...   
3316  test/2295_jpg.rf.a7f90255600bae464023779db2cd5...   
3317  test/2295_jpg.rf.a7f90255600bae464023779db2cd5...   
3318  test/2295_jpg.rf.a7f90255600bae464023779db2cd5...   
3319  test/2295_jpg.rf.a7f90255600bae464023779db2cd5...   
3320  test/2295_jpg.rf.a7f90255600bae464023779db2cd5...   

                                               features  label  
0     [0.2640417780831969, 0.2640417780831969, 0.0, ...      1  
1     [0.29242355241801, 0.0, 0.0, 0.0, 0.1561207866...      1  
2     [0.08204885055352479, 0.0, 0.17405188797561302...      1  
3     [0.4397735461528512, 0.0,

In [28]:
# Load mô hình
rf_model = joblib.load("random_forest_model.pkl")
scaler = joblib.load("scaler.pkl")

# Thư mục lưu kết quả
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

unique_test_df = test_df.drop_duplicates(subset=["image_path"])

# Duyệt qua tập test
for index, row in unique_test_df.iterrows():
    image_path = row["image_path"]
    detected_boxes = detect_people(image_path, rf_model, scaler)

    img = cv2.imread(image_path)
    if img is not None:
        for (x1, y1, x2, y2) in detected_boxes:
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        output_path = os.path.join(output_folder, os.path.basename(image_path))
        cv2.imwrite(output_path, img)
        print(f"✅ Đã lưu ảnh kết quả: {output_path}")
    else:
        print(f"❌ Lỗi khi đọc ảnh: {image_path}")

print("🎯 Hoàn thành nhận diện trên tập test!")


✅ Đã lưu ảnh kết quả: output\11516_jpg.rf.7cf4e332eafdb31672c0025195b6c445.jpg
✅ Đã lưu ảnh kết quả: output\12545_jpg.rf.69333f3b7d11b9cce33062b0cacfd523.jpg
✅ Đã lưu ảnh kết quả: output\6552_jpg.rf.ad79089b0fb56be23f73253fc5c0a921.jpg
✅ Đã lưu ảnh kết quả: output\13876_jpg.rf.430b135cb2913a6d1672ab657b952b1f.jpg
✅ Đã lưu ảnh kết quả: output\5414_jpg.rf.b212d2891d8ba3d586fd3dd2884568c1.jpg
✅ Đã lưu ảnh kết quả: output\8008_jpg.rf.4a6020858202d28a1444b6cd222183f5.jpg
✅ Đã lưu ảnh kết quả: output\2311_jpg.rf.b7409c0ef73d2082c68e291f8433913f.jpg
✅ Đã lưu ảnh kết quả: output\13489_jpg.rf.c8cbd2afc7201a20e4ee39bea97ffdc6.jpg
✅ Đã lưu ảnh kết quả: output\13116_jpg.rf.cf2f2d6c4477e608a578399a74c4a528.jpg
✅ Đã lưu ảnh kết quả: output\5516_jpg.rf.4f3152a305fe00760291bb221a00fb7e.jpg
✅ Đã lưu ảnh kết quả: output\9007_jpg.rf.7f574479ef4706adb3601992cf48b7d0.jpg
✅ Đã lưu ảnh kết quả: output\11449_jpg.rf.ab35b7bedca12232dc1658be81fd4616.jpg
✅ Đã lưu ảnh kết quả: output\6587_jpg.rf.389aee3ce0a6a0793

KeyboardInterrupt: 